In [118]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

In [119]:
FILEIN = 'midterm_1_data.xlsx'
sheet_exrets = 'excess returns'
sheet_spy = 'spy'

retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')

In [120]:
spy.head()

,SPY
date,
2016-01-15,-0.021430
2016-01-22,0.014429
2016-01-29,0.016801
2016-02-05,-0.029789
2016-02-12,-0.007023


In [121]:
retsx.head()

,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,XOM
date,,,,,,,
2016-01-15,-0.006077,-0.033445,-0.068591,-0.092883,-0.035777,-0.036346,0.030855
2016-01-22,0.045278,0.026605,0.047061,0.050539,0.050316,-0.010817,-0.011908
2016-01-29,-0.051720,0.042053,-0.027223,0.018024,0.009834,-0.067483,0.005221
2016-02-05,-0.036236,-0.096825,-0.151901,-0.104974,-0.082989,-0.156939,0.021309
2016-02-12,-0.007887,-0.000785,0.002275,-0.034052,-0.003101,-0.078688,0.013486


In [122]:
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats


summary_stats_retsx = performance_summary(retsx, 52)
summary_stats_retsx

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
AAPL,0.319421,0.283883,1.125183,-0.334342,2.672198,-0.052313,-0.085612,-0.190566,0.143562,-0.372094,2018-10-05,2019-01-04,2019-11-08
MSFT,0.288087,0.240206,1.199334,-0.359175,1.737189,-0.049366,-0.071559,-0.150492,0.104231,-0.299537,2020-02-14,2020-03-20,2020-07-03
AMZN,0.239457,0.310389,0.771474,-0.210630,1.746315,-0.061868,-0.096065,-0.151901,0.156111,-0.468127,2021-07-09,2023-01-06,NaT
NVDA,0.650658,0.468096,1.390011,0.425676,2.244417,-0.083805,-0.119446,-0.210199,0.332580,-0.592344,2021-11-19,2022-10-14,2023-05-19
GOOGL,0.193328,0.274217,0.705020,0.041986,1.143573,-0.055729,-0.078408,-0.135524,0.149258,-0.348297,2022-03-25,2023-01-06,NaT
TSLA,0.569728,0.607026,0.938556,0.441455,1.527376,-0.122519,-0.155313,-0.284957,0.334897,-0.682185,2021-11-05,2023-01-06,NaT
XOM,0.124196,0.311613,0.398557,0.097936,3.129459,-0.061685,-0.097340,-0.175338,0.184173,-0.671435,2016-12-16,2020-03-20,2022-03-11


In [123]:
max_sharpe_ticker = summary_stats_retsx['Sharpe Ratio'].idxmax()
max_sharpe_value = summary_stats_retsx['Sharpe Ratio'].max()
min_sharpe_ticker = summary_stats_retsx['Sharpe Ratio'].idxmin()
min_sharpe_value = summary_stats_retsx['Sharpe Ratio'].min()

print(f"The highest Sharpe ratio is {max_sharpe_value:.2f}, which comes from {max_sharpe_ticker}.")
print(f"The lowest Sharpe ratio is {min_sharpe_value:.2f}, which comes from {min_sharpe_ticker}.")

The highest Sharpe ratio is 1.39, which comes from NVDA.
The lowest Sharpe ratio is 0.40, which comes from XOM.


In [131]:
def tangency_weights(returns, cov_mat = 1):
    
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov()*12))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat*12))  
        
    # ones = np.ones(returns.columns[1:].shape) 
    # mu = returns.mean()*12
    # scaling = 1/(np.transpose(ones) @ cov_inv @ mu)



    ones = np.ones((returns.shape[1], 1))
    mu = returns.mean().values.reshape(-1, 1)  # Convert mu to a column vector

    print("ones shape:", ones.shape)
    print("cov_inv shape:", cov_inv.shape)
    print("mu shape:", mu.shape)
    
    scaling = 1 / (ones.T @ cov_inv @ mu)  # Transpose ones to make it a row vector
    tangent_return = scaling*(cov_inv @ mu) 
    tangency_wts = pd.DataFrame(index = returns.columns[:], data = tangent_return[:], columns = ['Tangent Weights'] )
        
    return tangency_wts

w_t = tangency_weights(retsx.reset_index(drop=True), cov_mat=1)
w_t

ones shape: (7, 1)
cov_inv shape: (7, 7)
mu shape: (7, 1)


,Tangent Weights
AAPL,0.322605
MSFT,0.787496
AMZN,-0.228607
NVDA,0.495996
GOOGL,-0.502721
TSLA,0.105975
XOM,0.019257


In [132]:
w_tan_summary_statistics = performance_summary(retsx @ w_t , 52)
w_tan_summary_statistics

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
Tangent Weights,0.563474,0.358351,1.572409,0.00438,1.899066,-0.063723,-0.096174,-0.223741,0.193774,-0.383571,2018-09-14,2019-01-04,2019-11-08
